# A Function Example
> *Let’s take a look at how a function call works in a real program.*

We will write the `power` function:
- Two parameters: the number and the power we want to raise it to.
- Ex: 2 and 3 = 2*2*2 = 8
- To simplify, we'll only allow numbers 1 and greater.

Some changes since 2003/2004:
- We live in a 64-bit dominant era now.
- `Error: invalid instruction suffix for push/pop`
    - Fix: <https://stackoverflow.com/a/21680219/13973029>
- `Error: operand type mismatch for pop/push`
    - Fix: <https://stackoverflow.com/a/29573784/13973029>

Notes:
- 64-bit registers (rax, rbx, etc.) use 8-byte words instead of 4-byte ones (unlike eax, ebx, etc.). And, so does the stack. So, compensate for that.
- An explanation of the stack's base pointer register:
    - Note: <https://stackoverflow.com/questions/1395591/what-is-exactly-the-base-pointer-and-stack-pointer-to-what-do-they-point>

In [ ]:
# PURPOSE:  A program to illustrate how functions work by computing the 
#           value of 2^3 + 5^2. 

.section .data
# Everything in the main program is stored in registers so the data section doesn't have anything.

.section .text
.globl _start
_start:
    # Call 'power' subroutines to do 2**3: 
    push $3         # (2) power
    push $2         # (1) base number
    call power      # transfer execution
    add $16, %rsp   # reset stack pointer

    push %eax   # save 2**3 for later

    # Call 'power' subroutines to do 5**2:    
    push $2         # (2) power
    push $5         # (1) base number
    call power      # transfer execution
    add $16, %rsp   # reset stack pointer

    pop %rbx        # get 2**3
    add %rax, %rbx  # store & print 5**3 + 2**3

    # Exit properly:
    mov $1, %rax
    int $0x80


# PURPOSE:  A function to compute the value of a number raised to 
#           a power.
# INPUT:    
#           (1) The base number.
#           (2) The power to raise the base up to.
# OUTPUT:   Returns the result value.
# NOTES:    The power must be 1 or greater.
# VARIABLES:
#           %rbx holds the base number.
#           %rcx holds the power.
#           -8(%rbp) holds the current result.
#           %rax is temporary storage + return register.
power:
    # Preserve previous frame pointer:
    push %rbp

    # Use the current frame pointer:
    mov %rsp, %rbp

    # Allocate 8 bytes for local storage for on variable:
    sub $8, %rsp

    # Assign arguments to high-speed registers:
    mov 16(%rbp), %rbx
    mov 24(%rbp), %rcx

    # Initialize local variable to store current result temporarily:
    mov %rbx, -8(%rbp)

power_loop_start:
    # Exit condition (powering exhausted from n>=1 to 1):
    cmp $1, %rcx
    je end_power

    # Multiply the current result with the base number:
    imul %rbx, %rax

    # Consume a power:
    dec %rcx

    # Repeat iteration:
    jmp power_loop_start

end_power:
    # Restore the stack pointer:
    mov %rbp, %rsp

    # Restore previous frame pointer:
    pop %rbp

    # Return control back to caller:
    ret

...

- Save important register values to the stack because those registers will be used.
- Provide documentation for the function; how to interface with it.

...

The following tells the linker to treat the `power` symbol as a function. 
```assembly
.type power, @function
```

Afterwards, the `power` label gets defined:
```assembly
power:
```
to the address the instructions following it begin. This is how `call` works. Compared to `jmp`, `call` pushes the return address onto the stack so that the function can return.

Next, the following instructions set up the function:
```assembly
push %rbp
mov %rsp, %rbp
sub $8, %rsp
```
At this point, the stack looks like:
```
Base Number    <--- 24(%rbp)
Power          <--- 16(%rbp)
Return Address <--- 8(%rbp)
Old %rbp       <--- (%rbp)
Current result <--- -8(%rbp) and (%rsp)
```
Local variables are useful when this function needs to call another and needs to access those values.

> *A good project to try now is to extend the program so it will return the value of a number if the power is 0 (hint, anything raised to the zero power is 1). Keep trying. If it doesn’t work at first, try going through your program by hand with a scrap of paper, keeping track of where `%ebp` and `%esp` are pointing, what is on the stack, and what the values are in each register.*
- [./my_power.s](./my_power.s)